# Dev for expressions of quotient derivative

Yuri Shimane, 2021.11.26

In [132]:
import numpy as np
import sympy as sym
import math
from sympy import lambdify

In [49]:
# define parameters
rpmin, k_petro, m_petro, n_petro, r_petro, b_petro = sym.symbols("rpmin k_petro m_petro n_petro r_petro b_petro")
wp = sym.symbols("Wp")
mu = sym.symbols("mu")
f = sym.symbols("f")

In [20]:
# orbital elements
a, e, i, ra, om, ta = sym.symbols("a e i Omega omega theta")

# targeted orbital elements
aT, eT, iT, raT, omT, taT = sym.symbols("a_T e_T i_T Omega_T omega_T theta_T")

# weights on orbital elements
wa, we, wi, wra, wom, wta = sym.symbols("w_a w_e w_i w_Omega w_omega w_theta")

In [27]:
oe = [a,e,i,ra,om,ta]
oe

[a, e, i, Omega, omega, theta]

In [34]:
oeT = [aT, eT, iT, raT, omT, taT]
oeT

[a_T, e_T, i_T, Omega_T, omega_T, theta_T]

In [52]:
woe = [wa, we, wi, wra, wom, wta]
woe

[w_a, w_e, w_i, w_Omega, w_omega, w_theta]

In [137]:
phi1, phi2 = sym.symbols("phi1 phi2")

In [138]:
def angle_difference(phi1, phi2):
    return sym.acos(sym.cos(phi1 - phi2))

In [156]:
expr = sym.sin(i) + sym.cos(i)

In [157]:
foo = lambdify(i, expr)#, 'numpy')

In [155]:
foo(0.29)

1.2441961006175326

In [54]:
doe = [a-aT, e-eT, i-iT, 
       sym.acos(sym.cos(ra - raT)),
       sym.acos(sym.cos(om - omT)),
       sym.acos(sym.cos(ta - taT)),
      ]

In [160]:
fun_doe = lambdify([a,e,i,ra,om,ta,aT,eT,iT,raT,omT,taT], doe)  # function version

In [209]:
def quotient(mu, f, oe, oeT, rpmin, m_petro, n_petro, r_petro, b_petro, wp, woe):
    # unpack elements
    a,e,i,ra,om,ta = oe
    aT, eT, iT, raT, omT, taT = oeT
    rp = a*(1-e)
    p = a*(1 - e**2)
    h = sym.sqrt(a*mu*(1-e**2))
    doe = [a-aT, e-eT, i-iT, 
           sym.acos(sym.cos(ra - raT)),
           sym.acos(sym.cos(om - omT)),
           sym.acos(sym.cos(ta - taT)),
          ]

    # compute omdot
    radot = p*f/(h*sym.sin(i)*(sym.sqrt(1 - e**2*sym.cos(om)**2) - e*abs(sym.sin(om))))
    cos_ta_xx = ((1-e**2)/(2*e**3)+sym.sqrt(0.25*((1-e)**2/e**3)**2 + 1/27))**(1/3) \
                - (-(1-e**2)/(2*e**3)+sym.sqrt(0.25*((1-e)**2/e**3)**2 + 1/27))**(1/3) - 1/e
    r_xx = p/(1+e*cos_ta_xx)
    omdot_i = f/(e*h)*sym.sqrt(p**2*cos_ta_xx**2 + (p+r_xx)**2*(1-cos_ta_xx**2))
    omdot_o = radot*abs(sym.cos(i))
    omdot = (omdot_i + b_petro*omdot_o)/(1+b_petro)

    # compute oedot
    oedot = [
        2*f*sym.sqrt(a**3 * (1+e)/(mu*(1-e))),
        2*p*f/h,
        p*f/(h*(sym.sqrt(1 - e**2*sym.sin(om)**2) - e*abs(sym.cos(om)))),
        radot,
        omdot,
    ]
    
    # compute periapsis radius constraint P
    p_rp = sym.exp(k*(1.0 - rp/rpmin))
    # compute scaling for each element Soe
    soe = [(1 + ((a-aT)/(m_petro*aT))**n_petro)**(1/r_petro), 1.0, 1.0, 1.0, 1.0, 1.0]
    # compute quotient Q
    q = (1 + wp*p_rp)
    for idx in range(5):
        q = q * woe[idx]*soe[idx]*(doe[idx]/oedot[idx])**2
    fun_q = lambdify([mu, f, oe, oeT, rpmin, m_petro, n_petro, r_petro, b_petro, wp, woe], q, "numpy")
    return q, fun_q

In [210]:
q, fun_q = quotient(mu, f, oe, oeT, rpmin, m_petro, n_petro, r_petro, b_petro, wp, woe)

In [211]:
fun_q

<function _lambdifygenerated(mu, f, _Dummy_41, _Dummy_42, rpmin, m_petro, n_petro, r_petro, b_petro, Wp, _Dummy_43)>

In [212]:
import math

In [213]:
def gauss_var_matrix(mu,oe):
     # unpack elements
    a,e,i,ra,om,ta = oe
    rp = a*(1-e)
    p = a*(1 - e**2)
    h = sym.sqrt(a*mu*(1-e**2))
    r = h**2/(mu*(1+e*sym.cos(ta)))
    # let psi be column major!
    psi = [
        [
            2*a**2/h*e*sym.sin(ta),
            p/h*sym.sin(ta),
            0.0,
            0.0,
            -p/(e*h)*sym.cos(ta)
        ], 
        [
            2*a**2/h * p/r,
            (p+r)/h*sym.cos(ta) + r*e,
            0.0,
            0.0,
            (p+r)/(e*h)*sym.sin(ta)
        ], 
        [
            0.0,
            0.0,
            r*sym.cos(ta+om)/h,
            r*sym.sin(ta+om)/(h*sym.sin(i)),
            -r*sym.sin(ta+om)*sym.cos(i)/(h*sym.sin(i)),
        ]
    ]
    fun_psi = lambdify([mu,oe],psi,"numpy")
    return psi, fun_psi

In [214]:
psi, fun_psi = gauss_var_matrix(mu,oe)

In [215]:
psi[0]

[2*a**2*e*sin(theta)/sqrt(a*mu*(1 - e**2)),
 a*(1 - e**2)*sin(theta)/sqrt(a*mu*(1 - e**2)),
 0.0,
 0.0,
 -a*(1 - e**2)*cos(theta)/(e*sqrt(a*mu*(1 - e**2)))]

In [216]:
fun_psi

<function _lambdifygenerated(mu, _Dummy_44)>

In [217]:
def thrust_direction(q, oe):
    dqdoe = []
    for idx in range(5):
        dqdoe.append(sym.diff(q, oe[idx]))
    psi = gauss_var_matrix(mu,oe)
    # multiply
    ux = -(psi[0][0]*dqdoe[0] + psi[0][1]*dqdoe[1] + psi[0][2]*dqdoe[2] + psi[0][3]*dqdoe[3] + psi[0][4]*dqdoe[4])
    uy = -(psi[1][0]*dqdoe[0] + psi[1][1]*dqdoe[1] + psi[1][2]*dqdoe[2] + psi[1][3]*dqdoe[3] + psi[1][4]*dqdoe[4])
    uz = -(psi[2][0]*dqdoe[0] + psi[2][1]*dqdoe[1] + psi[2][2]*dqdoe[2] + psi[2][3]*dqdoe[3] + psi[2][4]*dqdoe[4])
    fun_ux = lambdify([q,oe], ux)
    fun_uy = lambdify([q,oe], uy)
    fun_uz = lambdify([q,oe], uz)
    return [ux,uy,uz], [fun_ux, fun_uy, fun_uz]

In [218]:
us, fun_us = thrust_direction(q, oe)

TypeError: can't multiply sequence by non-int of type 'Add'

In [219]:
lambdify([q,oe], thrust_direction, 'numpy')

SympifyError: Sympify of expression 'could not parse '<function thrust_direction at 0x000001B14DDA1DC0>'' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 1)